In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# load dataset
df = pd.read_csv(
    "/content/drive/MyDrive/cs3244/dataset/train-balanced.csv.bz2",
    compression="bz2",
    sep="\t",
    header=None
)

# assign column names for clarity
df.columns = [
    "label", "text", "author", "subreddit",
    "upvotes", "downvotes", "score",
    "date", "timestamp", "comment"
]


In [ ]:
# text cleaning
def clean_text(text):
    if not isinstance(text, str):
        return ""
    return text.strip()

df["text"] = df["text"].astype(str).apply(clean_text)

In [ ]:
# feature extraction functions
def count_uppercase_ratio(text):
    if len(text) == 0:
        return 0
    upper = sum(1 for c in text if c.isupper())
    return upper / len(text)

def count_exclamations(text):
    return text.count("!")

def count_questions(text):
    return text.count("?")

def count_ellipsis(text):
    return text.count("...")

def count_elongated_words(text):
    # words with 3 or more repeated characters
    return len(re.findall(r"(.)\1{2,}", text.lower()))

def count_quoted_words(text):
    return len(re.findall(r"\"[^\"]+\"|'[^']+'", text))

def has_hashtag_sarcasm(text):
    return int(bool(re.search(r"#sarcasm|/s", text.lower())))

def avg_word_len(text):
    words = re.findall(r"\w+", text)
    return sum(len(w) for w in words) / len(words) if words else 0

In [ ]:
# apply features
df["caps_ratio"] = df["text"].apply(count_uppercase_ratio)
df["excl_count"] = df["text"].apply(count_exclamations)
df["quest_count"] = df["text"].apply(count_questions)
df["ellipsis_count"] = df["text"].apply(count_ellipsis)
df["elongation_count"] = df["text"].apply(count_elongated_words)
df["quoted_words"] = df["text"].apply(count_quoted_words)
df["sarcasm_marker"] = df["text"].apply(has_hashtag_sarcasm)
df["avg_word_len"] = df["text"].apply(avg_word_len)

print(df.head())

   label                                               text        author  \
0      0                                   Dang dog, thanks    Mattys1174   
1      0  to summon the powers of the flying spaghetti m...    yugiohhero   
2      0       i did that 3rd last 1 by accident last night    yugiohhero   
3      0  He's insane, used him in DC, better than Blake...  masternater6   
4      0  Forgot about him, he's a pretty pointless card...  masternater6   

         subreddit  upvotes  downvotes  score     date   timestamp  \
0           NHLHUT        1         -1     -1  2017-03  1490927299   
1    CringeAnarchy        1         -1     -1  2017-02  1486349313   
2  TownofSalemgame        8         -1     -1  2017-03  1488514232   
3           NHLHUT        1         -1     -1  2017-02  1486097950   
4           NHLHUT        1         -1     -1  2017-04  1491597507   

                                             comment  caps_ratio  excl_count  \
0                             Great 

In [ ]:
# TF-IDF lexical features
# Word n-grams capture token sequences like “yeah right”
word_vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=10000,
    lowercase=True,
    stop_words='english'
)
X_word_tfidf = word_vectorizer.fit_transform(df["text"])

# Character n-grams capture patterns like “!!!” or “soooo”
char_vectorizer = TfidfVectorizer(
    analyzer='char',
    ngram_range=(3, 5),
    max_features=5000
)
X_char_tfidf = char_vectorizer.fit_transform(df["text"])

print("Word TF-IDF shape:", X_word_tfidf.shape)
print("Char TF-IDF shape:", X_char_tfidf.shape)

Word TF-IDF shape: (1125678, 10000)
Char TF-IDF shape: (1125678, 5000)


In [ ]:
# combine engineered + TF-IDF features later for ML + logistic regression model
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_extra = df[["caps_ratio","excl_count","quest_count","ellipsis_count",
              "elongation_count","quoted_words","sarcasm_marker","avg_word_len"]].values
X_full = hstack([X_word_tfidf, X_char_tfidf, X_extra])

X_train, X_test, y_train, y_test = train_test_split(
    X_full, df["label"], test_size=0.2, random_state=42
)

model = LogisticRegression(max_iter=5000)
model.fit(X_train, y_train)
print("Validation accuracy:", model.score(X_test, y_test))

Validation accuracy: 0.7144259469831569


In [ ]:
import joblib

# save df with engineered features
output_csv = "sarcasm_features.csv"
df.to_csv(output_csv, index=False)
print(f"Saved feature dataframe to {output_csv}")

# save trained model
model_file = "sarcasm_logreg_model.pkl"
joblib.dump(model, model_file)
print(f"Saved trained model to {model_file}")

# save TF-IDF vectorizers (for future inference)
joblib.dump(word_vectorizer, "word_vectorizer.pkl")
joblib.dump(char_vectorizer, "char_vectorizer.pkl")
print("Saved vectorizers (word & char TF-IDF).")


Saved feature dataframe to sarcasm_features.csv
Saved trained model to sarcasm_logreg_model.pkl
Saved vectorizers (word & char TF-IDF).
